<a href="https://colab.research.google.com/github/Zywia/AD-projekt-2-/blob/master/projekt_2_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2

In [2]:
!pip install tf-models-nightly

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pickle
# from bert.tokenization
from bert import bert_tokenization
import official.nlp.bert.tokenization
# from bert.tokenization import FullSentencePieceTokenizer


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/1",
                            trainable=True)

In [6]:
train_data, test_data = tfds.load(name="civil_comments", split=["train", "test"], data_dir="/content/drive/My Drive/AD/civil_comments", batch_size=-1, as_supervised=True)



In [7]:
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [8]:
negative_comments_positions = np.argwhere(train_labels == 1)[:,0]
not_all_positive_comments_positions = np.argwhere(train_labels == 0)[:negative_comments_positions.shape[0], 0]

negative_comments_positions_test = np.argwhere(test_labels == 1)[:,0]
not_all_positive_comments_positions_test = np.argwhere(test_labels == 0)[:negative_comments_positions_test.shape[0], 0]


not_all_positive_comments_positions_test.shape

(222,)

In [9]:
a = np.hstack ( (not_all_positive_comments_positions, negative_comments_positions))
b = np.hstack ( (not_all_positive_comments_positions_test, negative_comments_positions_test))
a = np.random.permutation(a)
b = np.random.permutation(b)
train_examples = train_examples[a]
train_labels = train_labels[a]

test_examples = test_examples[b]
test_labels = test_labels[b]


In [10]:
test_examples.shape

(444,)

In [11]:
?official.nlp.bert.tokenization.FullSentencePieceTokenizer

In [12]:
sp_model_file = bert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = official.nlp.bert.tokenization.FullSentencePieceTokenizer(sp_model_file)


def tokenize_text(txt):
    tokenized_txt = tokenizer.tokenize(txt)
    tokenized_txt = ["[CLS]"] + tokenized_txt + ["[SEP]"]
    tokenized_txt = tokenizer.convert_tokens_to_ids(tokenized_txt)
    return tokenized_txt

In [13]:

train_examples = list(map(tokenize_text, train_examples))
test_examples = list(map(tokenize_text, test_examples))

In [46]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_examples, maxlen=100, padding='post')
max_seq_length = x_train.shape[1]
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_examples, maxlen=max_seq_length, padding='post', truncating='post')

train_mask = (x_train != 0).astype(int)
test_mask = (x_test != 0).astype(int)

train_segments = np.zeros(x_train.shape)
test_segments = np.zeros(x_test.shape)

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_inputs = [input_word_ids, input_mask, segment_ids]
pooled_output, sequence_output = bert_layer(bert_inputs)

final_layer = tf.keras.layers.Dense(1, activation="sigmoid")
final_output = final_layer(pooled_output)
# tf.keras.layers.


In [47]:
model = tf.keras.models.Model(inputs=bert_inputs, outputs=final_output)

In [48]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
              metrics=["accuracy"])

In [52]:
model.fit([x_train, train_mask, train_segments], train_labels, epochs=10, batch_size=32,
          validation_data=([x_test, test_mask, test_segments], test_labels))

Epoch 1/10
276/276 [==============================] - 301s 1s/step - loss: 0.3436 - accuracy: 0.8669 - val_loss: 0.4032 - val_accuracy: 0.8356
Epoch 2/10
276/276 [==============================] - 302s 1s/step - loss: 0.2305 - accuracy: 0.9197 - val_loss: 0.4125 - val_accuracy: 0.8153
Epoch 3/10
276/276 [==============================] - 302s 1s/step - loss: 0.1575 - accuracy: 0.9525 - val_loss: 0.4608 - val_accuracy: 0.8176
Epoch 4/10
276/276 [==============================] - 302s 1s/step - loss: 0.1009 - accuracy: 0.9737 - val_loss: 0.4998 - val_accuracy: 0.8063
Epoch 5/10
276/276 [==============================] - 302s 1s/step - loss: 0.0772 - accuracy: 0.9798 - val_loss: 0.7913 - val_accuracy: 0.8311
Epoch 6/10
276/276 [==============================] - 302s 1s/step - loss: 0.0591 - accuracy: 0.9840 - val_loss: 0.8521 - val_accuracy: 0.8086
Epoch 7/10
276/276 [==============================] - 302s 1s/step - loss: 0.0363 - accuracy: 0.9918 - val_loss: 0.5814 - val_accuracy: 0.8266

In [53]:
import pandas as pd

text = pd.read_csv("notI.csv", names=["sentance"], index_col=False)
text = text.to_numpy()

print (text[:,0].tolist())
text = text[:,0].tolist()
text = list(map(tokenize_text, text))
print(text[0])
text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=100, padding='post')
mask = (text != 0).astype(int)
segments = np.zeros(text.shape)


['temporrytext that is absoulutlely not important', 'anotehre aboulutally not important text', 'you are ugly so ugly', 'hate speech']
[2, 14935, 139, 622, 11969, 30, 25, 12178, 111, 8331, 5866, 102, 52, 681, 3]


In [55]:
model.save("/content/drive/My Drive/AD/check/my_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model/assets


In [58]:
m = tf.keras.models.load_model("/content/drive/My Drive/AD/check/my_model")
# m([text, mask, segments])


<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.00341263],
       [0.00178851],
       [0.996042  ],
       [0.89366364]], dtype=float32)>

In [71]:
resoult = m([text, mask, segments])

In [72]:
resoult = resoult > 0.5
resoult = np.array(resoult, dtype='int')

In [78]:
np.savetxt("foo.csv", resoult, delimiter=",")